In [ ]:

#    Прочитать про методы оптимизации для нейронных сетей https://habr.com/post/318970/
#    Реализовать самостоятельно логистическую регрессию
#        Обучить ее методом градиентного спуска
#        Методом nesterov momentum
#        Методом rmsprop
#    В качестве dataset'а взять Iris, оставив 2 класса:
#        Iris Versicolor
#        Iris Virginica



In [2]:
import pandas as pd
import numpy as np


In [3]:
from sklearn.datasets import load_iris
data_all = load_iris(True)
X = data_all[0]
y = data_all[1]


In [4]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [ ]:
# Обычнвая линейная регрессия

In [5]:
N = 100
X = np.random.uniform(low=0, high=100, size=(N,2))
Y = 1 + 0.3*X[:,0] + 1.5*X[:,1] +  np.random.normal(scale=5, size=N)

In [6]:
i = 0
m = 3
teta_old = np.zeros(m)
teta = np.ones(m)
eps = teta*0.00000001
alpha = 0.00000001

while (np.sum(np.abs(teta-teta_old) > eps)>0):
    teta_old = teta.copy()
    teta[0] = teta_old[0] - alpha * np.sum(np.dot(teta_old[1:],X.T)+teta_old[0] - Y)/len(Y)
    teta[1:] = teta_old[1:] - alpha*np.sum(X.T*(np.dot(teta_old[1:],X.T)+teta_old[0] - Y), axis=1)/len(Y)
    if(i%50000 == 0):
        print(i, teta-teta_old)
    i+=1 

print('Результат:', teta)

0 [-1.29764708e-07 -1.26611137e-05 -2.15117504e-06]
50000 [-6.03380679e-09 -3.43897495e-06  2.97560111e-06]
100000 [ 2.58220112e-11 -2.01227536e-06  2.22987748e-06]
150000 [-2.10410245e-10 -1.32548050e-06  1.49996926e-06]
200000 [-6.06999895e-10 -8.82522349e-07  1.00047130e-06]
250000 [-8.84664009e-10 -5.88127615e-07  6.66838343e-07]
300000 [-1.07043352e-09 -3.91966568e-07  4.44438879e-07]
350000 [-1.19425048e-09 -2.61232344e-07  2.96212484e-07]
400000 [-1.27673794e-09 -1.74101144e-07  1.97422882e-07]
450000 [-1.33167810e-09 -1.16030218e-07  1.31581884e-07]
500000 [-1.36825840e-09 -7.73272859e-08  8.77003776e-08]
550000 [-1.39260237e-09 -5.15326747e-08  5.84543651e-08]
600000 [-1.40879097e-09 -3.43411617e-08  3.89625721e-08]
650000 [-1.41954437e-09 -2.28834152e-08  2.59717414e-08]
700000 [-1.42667522e-09 -1.52470918e-08  1.73136521e-08]
750000 [-1.43139178e-09 -1.01576587e-08  1.15432348e-08]
Результат: [0.99711566 0.30794517 1.4984227 ]
